In [38]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import json,re,random
ps = PorterStemmer()
lz = WordNetLemmatizer()



In [25]:
sample_file = './reviewSamples20.json'

with open(sample_file,'r') as f:
    raw_data = f.read()

In [40]:
def process_raw_data(data_in_weird_format):
    data = data_in_weird_format.split('}\n')
    return [x+'}' for x in data if x != '']
data = process_raw_data(raw_data)
print(f'Processed {len(data)} lines of data')

_sample_data = json.loads(random.choice(data))
print('\n\nSample data')
print(json.dumps(_sample_data, indent=4, sort_keys=True))

Processed 20 lines of data


Sample data
{
    "business_id": "GdCRQU3VCh_x8fY84mbwYQ",
    "cool": 3,
    "date": "2011-02-13 17:46:55",
    "funny": 1,
    "review_id": "yMXmJzpEah6XfS_SSu920A",
    "stars": 5.0,
    "text": "Man, I love Toronto! Hiding in a strip mall on Overlea, find a dingy looking restaurant that serves up fantastic, cheap kabob...be warned regardless of when you come here you'll likely be waiting for a table as this place is always overrun with people...If you're a fan of perfectly grilled spicy meat, you must try this place out...now that I've been here I need to limit how many times I go here per month :)\n\nTake the drive out and have some kabob...if you don't own a car, borrow one or get a zipcar membership...totally worth it!",
    "useful": 1,
    "user_id": "MQq_1AlcDWsyNs7Ot8J4OQ"
}
